# Creating A New MLRun Project
  --------------------------------------------------------------------

creating a full project with multiple functions and workflow and working wit Git.

#### **notebook how-to's**
* Add local or library/remote functions
* Add a workflow
* Save to a remote git
* Run pipeline

<a id='top'></a>
#### **steps**
**[Add functions](#load-functions)**<br>
**[Create and save a workflow](#create-workflow)**<br>
**[Update remote git](#git-remote)**<br>
**[Run a pipeline workflow](#run-pipeline)**<br>

In [1]:
from mlrun import new_project, code_to_function

In [2]:
# update the dir and repo to reflect real locations
# the remote git repo must be initialized in GitHub
project_dir = "/User/new-proj"
remote_git = "https://github.com/<my-org>/<my-repo>.git"
newproj = new_project("new-project", project_dir, init_git=True)

Set the remote git repo and pull to sync in case it has some content

In [ ]:
newproj.create_remote(remote_git)

In [4]:
newproj.pull()

<a id='load-functions'></a>
### Load functions from remote URLs or marketplace
We create two functions:
1. Load a function from the function market (converted into a function object)
2. Create a function from file in the context dir (w copy a demo file into the dir) 

In [3]:
newproj.set_function("hub://load_dataset", "ingest").doc()

### Create a local function (use code from mlrun examples)

In [4]:
!curl -o {project_dir}/handler.py https://raw.githubusercontent.com/mlrun/mlrun/master/examples/handler.py

In [5]:
# add function with build config (base image, run command)
fn = code_to_function("tstfunc", filename="handler.py", kind="job")
fn.build_config(base_image="mlrun/mlrun", commands=["pip install pandas"])
newproj.set_function(fn)
print(newproj.func("tstfunc").to_yaml())

<a id='create-workflow'></a>
### Create a workflow file and store it in the context dir

In [6]:
%%writefile {project_dir}/workflow.py
from kfp import dsl
from mlrun import mount_v3io

funcs = {}

def init_functions(functions: dict, project=None, secrets=None):
    functions['ingest'].apply(mount_v3io())

@dsl.pipeline(
    name='demo project', description='Shows how to use mlrun project.'
)
def kfpipeline(p1=3):
    # first step build the function container
    builder = funcs['tstfunc'].deploy_step(with_mlrun=False)
    
    ingest = funcs['ingest'].as_step(name='load-data', params={'dataset': 'boston'})

    # first step
    s1 = funcs['tstfunc'].as_step(name='step-one', handler='my_func',
         image=builder.outputs['image'],
         params={'p1': p1})


In [7]:
newproj.set_workflow("main", "workflow.py")

In [8]:
print(newproj.to_yaml())

<a id='git-remote'></a>
### Register and push the project to a remote Repo

In [ ]:
newproj.push("master", "first push", add=["handler.py", "workflow.py"])

In [18]:
newproj.source

<a id='run-pipeline'></a>
### Run the workflow

In [9]:
newproj.run(
    "main",
    arguments={},
    artifact_path="v3io:///users/admin/mlrun/kfp/{{workflow.uid}}/",
)

**[back to top](#top)**